# Hacemos las importaciones oportunas

In [1]:
import asyncio

from joblib import Parallel, delayed

import requests as req
from bs4 import BeautifulSoup as bs

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.ui import Select

import time
import multiprocessing as mp

import pandas as pd
pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

from tqdm.notebook import tqdm

#from webdriver_manager.chrome import ChromeDriverManager
#PATH=ChromeDriverManager().install()

PATH=r'Chromedriver\chromedriver'

import warnings
warnings.filterwarnings('ignore')

# Abrir Archivo csv de los stats de los GM's (fuente: Kaggle)

In [2]:
stats_original = pd.read_csv(r'GM_players_statistics.csv')
stats = stats_original.copy()
stats.head()

,Unnamed: 0,name,username,profile,user_id,title,fide,country,followers,joined,last_online,current_rapid_rating,highest_rapid_rating,highest_rapid_date,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,highest_blitz_date,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,highest_bullet_date,bullet_win,bullet_draw,bullet_loss,highest_tactics_rating,highest_tactics_date,highest_puzzle_rush_score,is_streamer,status,league
0,0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,494,07/11/2015,17/09/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,07/11/2015,NaN,False,premium,NaN
1,1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,360,11/08/2017,13/07/2023,2383.0,2672.0,03/05/2020,12.0,5.0,6.0,2823.0,2914.0,14/07/2020,976.0,280.0,847.0,2661.0,2750.0,27/09/2021,32.0,8.0,47.0,3251,15/09/2021,49.0,False,premium,Bronze
2,2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,199,05/11/2014,08/12/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400,05/11/2014,NaN,False,premium,NaN
3,3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,142482,27/08/2010,15/07/2023,2925.0,2977.0,07/02/2023,43.0,23.0,8.0,3287.0,3322.0,21/06/2023,1402.0,292.0,334.0,3249.0,3390.0,28/03/2023,662.0,98.0,246.0,400,27/08/2010,NaN,False,premium,Elite
4,4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,14445,17/03/2013,16/07/2023,2794.0,3045.0,01/09/2019,122.0,103.0,46.0,3125.0,3136.0,28/05/2023,1012.0,194.0,322.0,3030.0,3113.0,30/11/2021,713.0,65.0,203.0,3519,14/06/2023,64.0,False,premium,Crystal


# Limpieza csv

In [3]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 1508 non-null   int64  
 1   name                       1241 non-null   object 
 2   username                   1508 non-null   object 
 3   profile                    1508 non-null   object 
 4   user_id                    1508 non-null   int64  
 5   title                      1508 non-null   object 
 6   fide                       913 non-null    float64
 7   country                    1508 non-null   object 
 8   followers                  1508 non-null   int64  
 9   joined                     1508 non-null   object 
 10  last_online                1508 non-null   object 
 11  current_rapid_rating       1055 non-null   float64
 12  highest_rapid_rating       997 non-null    float64
 13  highest_rapid_date         997 non-null    objec

In [4]:
stats = stats.drop(['is_streamer', 'status', 'league', 'highest_puzzle_rush_score', 'highest_tactics_date', 'highest_tactics_rating', 'joined', 'followers', 'last_online', 'highest_rapid_date', 'highest_blitz_date', 'highest_bullet_date'], axis=1)
stats

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,Komodo Engine,komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Vojtěch Plát,vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,PlayMagnus Carlsen,playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Magnus Carlsen,magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,Fabiano Caruana,fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,Stanojoski Zvonko,zona1,https://www.chess.com/member/Zona1,13526852,GM,NaN,North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,Dmitry Gurevich,zq1,https://www.chess.com/member/Zq1,4042329,GM,NaN,United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,Alexander Zubov,zubov_alexander,https://www.chess.com/member/Zubov_Alexander,83663714,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,Alexander Zubov,zubov_on_youtube,https://www.chess.com/member/Zubov_On_YouTube,148025331,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [5]:
stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1508 entries, 0 to 1507
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1508 non-null   int64  
 1   name                   1241 non-null   object 
 2   username               1508 non-null   object 
 3   profile                1508 non-null   object 
 4   user_id                1508 non-null   int64  
 5   title                  1508 non-null   object 
 6   fide                   913 non-null    float64
 7   country                1508 non-null   object 
 8   current_rapid_rating   1055 non-null   float64
 9   highest_rapid_rating   997 non-null    float64
 10  rapid_win              1055 non-null   float64
 11  rapid_draw             1055 non-null   float64
 12  rapid_loss             1055 non-null   float64
 13  current_blitz_rating   1388 non-null   float64
 14  highest_blitz_rating   1382 non-null   float64
 15  blit

In [6]:
mp.cpu_count() # nº de núcleos

4

In [7]:
stats['fide'].value_counts()

0.0       199
2500.0     22
2600.0     19
2550.0     14
2650.0     11
         ... 
2697.0      1
2698.0      1
2705.0      1
2706.0      1
3411.0      1
Name: fide, Length: 310, dtype: int64

### Partidas totales ganadas

In [8]:
rapid_win = stats['rapid_win'].tolist()
rapid_win[:5]

[nan, 12.0, nan, 43.0, 122.0]

In [9]:
blitz_win = stats['blitz_win'].tolist()
blitz_win[:5]

[nan, 976.0, nan, 1402.0, 1012.0]

In [10]:
total_wins = [x + y for x, y in zip(rapid_win, blitz_win)]
total_wins[:5]

[nan, 988.0, nan, 1445.0, 1134.0]

### Partidas totales empatadas

In [11]:
rapid_draw = stats['rapid_draw'].tolist()
rapid_draw[:5]

[nan, 5.0, nan, 23.0, 103.0]

In [12]:
blitz_draw = stats['blitz_draw'].tolist()
blitz_draw[:5]

[nan, 280.0, nan, 292.0, 194.0]

In [13]:
total_draws = [x + y for x, y in zip(rapid_draw, blitz_draw)]
total_draws[:5]

[nan, 285.0, nan, 315.0, 297.0]

### Partidas totales perdidas

In [14]:
rapid_loss = stats['rapid_loss'].tolist()
rapid_loss[:5]

[nan, 6.0, nan, 8.0, 46.0]

In [15]:
blitz_loss = stats['blitz_loss'].tolist()
blitz_loss[:5]

[nan, 847.0, nan, 334.0, 322.0]

In [16]:
total_losses = [x + y for x, y in zip(rapid_loss, blitz_loss)]
total_losses[:5]

[nan, 853.0, nan, 342.0, 368.0]

# Total_games

In [17]:
total_games = [x + y + z for x, y, z in zip(total_wins, total_draws, total_losses)]
total_games[:5]

[nan, 2126.0, nan, 2102.0, 1799.0]

## % wins_online

In [18]:
wins_online = [round((x / y)*100, 2) for x, y in zip(total_wins, total_games)]
wins_online[:5]

[nan, 46.47, nan, 68.74, 63.04]

## % draws_online

In [19]:
draws_online = [round((x / y)*100, 2) for x, y in zip(total_draws, total_games)]
draws_online[:5]

[nan, 13.41, nan, 14.99, 16.51]

## % losses_online

In [20]:
losses_online = [round((x / y)*100, 2) for x, y in zip(total_losses, total_games)]
losses_online[:5]

[nan, 40.12, nan, 16.27, 20.46]

# Lista de nombres necesria para escrapear

In [21]:
names = stats['name'].tolist()
names[:20]

['Komodo Engine',
 'Vojtěch Plát',
 'PlayMagnus Carlsen',
 'Magnus Carlsen',
 'Fabiano Caruana',
 'Iuri Shkuro',
 'Hikaru Nakamura',
 'Garry Kasparov',
 'Viswanathan Anand',
 'Levon Aronian',
 'Alireza Firouzja',
 'Hikaru Nakamura',
 'Alexander Grischuk',
 'Sergey Karjakin',
 'Maxime Vachier-Lagrave',
 'Ian Nepomniachtchi',
 'Anish Giri',
 'Shakhriyar Mamedyarov',
 'Timur Gareev',
 'Harikrishna Pentala']

In [22]:
def transformar_nombre(nombre_completo):
    if isinstance(nombre_completo, str):
        nombres = nombre_completo.split()
        apellido = nombres[-1]
        nombre = " ".join(nombres[:-1])
        return f"{apellido}, {nombre}"
    else:
        return nombre_completo

# Lista de nombres en formato "Nombre Apellido"
# names ya elaborada


In [23]:
# Transformar los nombres
nombres_transformados = list(map(transformar_nombre, names))

In [63]:
# Mostrar los nombres transformados
Name = []  # Aquí se almacenarán los nombres transformados

for nombre in nombres_transformados:
    Name.append(nombre)

print(Name)

['Engine, Komodo', 'Plát, Vojtěch', 'Carlsen, PlayMagnus', 'Carlsen, Magnus', 'Caruana, Fabiano', 'Shkuro, Iuri', 'Nakamura, Hikaru', 'Kasparov, Garry', 'Anand, Viswanathan', 'Aronian, Levon', 'Firouzja, Alireza', 'Nakamura, Hikaru', 'Grischuk, Alexander', 'Karjakin, Sergey', 'Vachier-Lagrave, Maxime', 'Nepomniachtchi, Ian', 'Giri, Anish', 'Mamedyarov, Shakhriyar', 'Gareev, Timur', 'Pentala, Harikrishna', 'Radjabov, Teimour', 'Giri, Anish', 'Kramnik, Vladimir', 'Perez, Leinier Dominguez', 'Onischuk, Alex', 'Zubov, Alexander', 'Wojtaszek, Radoslaw', 'Duda, Jan-Krzysztof', 'Yifan, Hou', 'Nakamura, Hikaru', 'Andreikin, Dmitry', 'Le, Liem', 'Maghsoodloo, Parham', 'Gujrathi, Vidit', 'Jobava, Baadur', 'Shtembuliak, Evgeny', 'Zherebukh, Yaroslav', 'Akobian, Varuzhan', 'Pons, Francisco Vallejo', 'Ipatov, Alexander', 'Durarbayli, Vasif', 'Esipenko, Andrey', 'Lenderman, Aleksandr', 'Naiditsch, Arkadij', 'Kovalev, Vladislav', 'Lenderman, Alex', 'Fedoseev, Vladimir', 'Ehlvest, Jaan', 'Deac, Bogdan

### Cambio la lista de nombres creada por la columna original del dataframe

In [67]:
stats['name'] = Name
stats

,Unnamed: 0,name,username,profile,user_id,title,fide,country,current_rapid_rating,highest_rapid_rating,rapid_win,rapid_draw,rapid_loss,current_blitz_rating,highest_blitz_rating,blitz_win,blitz_draw,blitz_loss,current_bullet_rating,highest_bullet_rating,bullet_win,bullet_draw,bullet_loss
0,0,"Engine, Komodo",komodochess,https://www.chess.com/member/KomodoChess,24944922,GM,3411.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"Plát, Vojtěch",vojtechplat,https://www.chess.com/member/VojtechPlat,37712368,GM,2950.0,Czech Republic,2383.0,2672.0,12.0,5.0,6.0,2823.0,2914.0,976.0,280.0,847.0,2661.0,2750.0,32.0,8.0,47.0
2,2,"Carlsen, PlayMagnus",playmagnus,https://www.chess.com/member/PlayMagnus,19578862,GM,2882.0,Norway,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,"Carlsen, Magnus",magnuscarlsen,https://www.chess.com/member/MagnusCarlsen,3889224,GM,2882.0,Norway,2925.0,2977.0,43.0,23.0,8.0,3287.0,3322.0,1402.0,292.0,334.0,3249.0,3390.0,662.0,98.0,246.0
4,4,"Caruana, Fabiano",fabianocaruana,https://www.chess.com/member/FabianoCaruana,11177810,GM,2835.0,United States,2794.0,3045.0,122.0,103.0,46.0,3125.0,3136.0,1012.0,194.0,322.0,3030.0,3113.0,713.0,65.0,203.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1503,1503,"Zvonko, Stanojoski",zona1,https://www.chess.com/member/Zona1,13526852,GM,NaN,North Macedonia,NaN,NaN,NaN,NaN,NaN,2375.0,2430.0,50.0,9.0,27.0,2102.0,2083.0,13.0,1.0,7.0
1504,1504,"Gurevich, Dmitry",zq1,https://www.chess.com/member/Zq1,4042329,GM,NaN,United States,2375.0,2555.0,7.0,2.0,6.0,2415.0,2675.0,3293.0,538.0,2521.0,2400.0,2594.0,496.0,47.0,402.0
1505,1505,"Zubov, Alexander",zubov_alexander,https://www.chess.com/member/Zubov_Alexander,83663714,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2961.0,2967.0,338.0,22.0,96.0,NaN,NaN,NaN,NaN,NaN
1506,1506,"Zubov, Alexander",zubov_on_youtube,https://www.chess.com/member/Zubov_On_YouTube,148025331,GM,NaN,Ukraine,NaN,NaN,NaN,NaN,NaN,2771.0,2763.0,138.0,3.0,11.0,NaN,NaN,NaN,NaN,NaN


In [75]:
stats.to_csv(r'stats_first_clean.csv', index=False)

# 1. Scrapeo de datos (rating)

## 1.1 Usando beautiful soup

In [26]:
url = 'https://2700chess.com/?per-page=100'

html = req.get(url).text

sopa = bs(html, 'html.parser')

In [27]:
#divs = for e in sopa.find_all('table')
#divs

In [28]:
#table = e.text for e in divs.find_all('table')

## 1.2 Usando Selenium

In [29]:
driver = webdriver.Chrome(PATH)

driver.get(url)

In [30]:
#cookies
aceptar = driver.find_element(By.XPATH, '//*[@id="accept-choices"]')

aceptar.click()

In [31]:
table = driver.find_element(By.XPATH, '//*[@id="live-ratings-table"]')
table

<selenium.webdriver.remote.webelement.WebElement (session="4125e1aa7abc0613d87581e7524479da", element="9B439C4FA38B3BF64AA6A0F68CB92EAE_element_105")>

In [32]:
rows = [[j.text for j in i.find_elements(By.TAG_NAME,'td')]
for i in driver.find_element(By.TAG_NAME, 'table').find_elements(By.TAG_NAME,'tr')]
rows

[[],
 ['1',
  '',
  'Carlsen, Magnus',
  '',
  '2838.4',
  '+3.4',
  '',
  '2825.0',
  '',
  '2887.0',
  '',
  '',
  '32',
  ''],
 ['2',
  '↑1',
  'Caruana, Fabiano',
  '',
  '2782.6',
  '+0.6',
  '',
  '2767.4',
  '',
  '2813.0',
  '',
  '',
  '31',
  ''],
 ['3',
  '↓1',
  'Nakamura, Hikaru',
  '',
  '2782.6',
  '−4.4',
  '',
  '2744.8',
  '',
  '2874.0',
  '',
  '',
  '35',
  ''],
 ['4',
  '',
  'Ding, Liren',
  '',
  '2780.0',
  '0.0',
  '',
  '2830.0',
  '',
  '2787 i',
  '',
  '',
  '30',
  ''],
 ['5',
  '↑1',
  'Firouzja, Alireza',
  '',
  '2777.0',
  '0.0',
  '',
  '2742.0',
  '',
  '2896.0',
  '',
  '',
  '20',
  ''],
 ['6',
  '↓1',
  'Nepomniachtchi, Ian',
  '',
  '2774.8',
  '−4.2',
  '',
  '2763.2',
  '',
  '2785.0',
  '',
  '',
  '33',
  ''],
 ['7',
  '↑1',
  'So, Wesley',
  '',
  '2760.6',
  '−8.4',
  '',
  '2783.2',
  '',
  '2765.0',
  '',
  '',
  '29',
  ''],
 ['8',
  '↓1',
  'Giri, Anish',
  '',
  '2759.6',
  '−9.4',
  '',
  '2696.2',
  '',
  '2751.6',
  '',
  '',
  '29

In [33]:
table = pd.DataFrame(rows)
table.columns = [i.text for i in driver.find_element(By.TAG_NAME, 'table').find_elements(By.TAG_NAME,'tr')[0].find_elements(By.TAG_NAME,'th')]
driver.quit()
table

,#,↑↓,Name,,Classic,+/−,,Rapid,,Blitz,,,Age,
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,,"Carlsen, Magnus",,2838.4,+3.4,,2825.0,,2887.0,,,32,
2,2,↑1,"Caruana, Fabiano",,2782.6,+0.6,,2767.4,,2813.0,,,31,
3,3,↓1,"Nakamura, Hikaru",,2782.6,−4.4,,2744.8,,2874.0,,,35,
4,4,,"Ding, Liren",,2780.0,0.0,,2830.0,,2787 i,,,30,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,,"Najer, Evgeniy",,2648.0,0.0,,2579.0,,2545.0,,,46,
97,97,↓14,"Saric, Ivan",,2647.5,−9.5,,2631.0,,2586.0,,,32,
98,98,,"Malakhov, Vladimir",,2646.0,0.0,,2666.0,,2652.0,,,42,
99,99,↓4,"Liang, Awonder",,2645.6,−3.4,,2518.4,,2538.0,,,20,


In [34]:
column_indices = [1, 3, 5, 6, 8, 10, 11, 13]
table = table.drop(table.columns[column_indices], axis=1)
table

,#,Name,Classic,Rapid,Blitz,Age
0,None,None,None,None,None,None
1,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
2,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
4,4,"Ding, Liren",2780.0,2830.0,2787 i,30
...,...,...,...,...,...,...
96,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46
97,97,"Saric, Ivan",2647.5,2631.0,2586.0,32
98,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42
99,99,"Liang, Awonder",2645.6,2518.4,2538.0,20


In [35]:
table = table.drop(table.index[0])
table

,#,Name,Classic,Rapid,Blitz,Age
1,1,"Carlsen, Magnus",2838.4,2825.0,2887.0,32
2,2,"Caruana, Fabiano",2782.6,2767.4,2813.0,31
3,3,"Nakamura, Hikaru",2782.6,2744.8,2874.0,35
4,4,"Ding, Liren",2780.0,2830.0,2787 i,30
5,5,"Firouzja, Alireza",2777.0,2742.0,2896.0,20
...,...,...,...,...,...,...
96,96,"Najer, Evgeniy",2648.0,2579.0,2545.0,46
97,97,"Saric, Ivan",2647.5,2631.0,2586.0,32
98,98,"Malakhov, Vladimir",2646.0,2666.0,2652.0,42
99,99,"Liang, Awonder",2645.6,2518.4,2538.0,20


In [36]:
table.to_csv(r'rating_top_100.csv', index=False)

### Crear una lista con los jugarores

In [37]:
top_100 = table['Name'].tolist()
print(top_100)


['Carlsen, Magnus', 'Caruana, Fabiano', 'Nakamura, Hikaru', 'Ding, Liren', 'Firouzja, Alireza', 'Nepomniachtchi, Ian', 'So, Wesley', 'Giri, Anish', 'Gukesh D', 'Anand, Viswanathan', 'Rapport, Richard', 'Karjakin, Sergey', 'Dominguez Perez, Leinier', 'Radjabov, Teimour', 'Aronian, Levon', 'Mamedyarov, Shakhriyar', 'Le, Quang Liem', 'Grischuk, Alexander', 'Duda, Jan-Krzysztof', 'Vachier-Lagrave, Maxime', 'Topalov, Veselin', 'Wei, Yi', 'Yu, Yangyi', 'Vitiugov, Nikita', 'Vidit, Santosh Gujrathi', 'Abdusattorov, Nodirbek', 'Praggnanandhaa R', 'Keymer, Vincent', 'Vallejo Pons, Francisco', 'Harikrishna, Pentala', 'Dubov, Daniil', 'Maghsoodloo, Parham', 'Sjugirov, Sanan', 'Erigaisi Arjun', 'Deac, Bogdan-Daniel', 'Robson, Ray', 'Wang, Hao', 'Van Foreest, Jorden', 'Artemiev, Vladislav', 'Sevian, Samuel', 'Shankland, Sam', 'Eljanov, Pavel', 'Tomashevsky, Evgeny', 'Xiong, Jeffery', 'Sargissian, Gabriel', 'Nihal Sarin', 'Svidler, Peter', 'Bu, Xiangzhi', 'Sarana, Alexey', 'Navara, David', 'Anton Gui

# Scrapeo de datos de 'Yottachess.com'

In [38]:
# ocultar ser un robot

from selenium.webdriver.chrome.options import Options

# opciones del driver
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

In [39]:
# entrando en la url

url = 'https://www.yottachess.com/'

driver = webdriver.Chrome(PATH, options=opciones)

driver.get(url)

time.sleep(2)

In [40]:
#aceptar cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')

aceptar.click()

time.sleep(2)

In [41]:
# introducir texto de busqueda, (habrá que hacer un loop para ir metiendo los nombres del top 100)

texto = driver.find_element(By.XPATH, '//*[@id="data"]')

texto.send_keys('Carlsen, Magnus')

time.sleep(2)

In [42]:
texto.send_keys(Keys.ENTER)   # aprieta el boton enter del teclado

### intento coger la tabla entera, (no ha funcionado por lo que cogerésolamete los datos que me interesan)

In [43]:
wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
wins

'2879 (48.53 %)'

In [44]:
draw = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
draw

'718 (35.27 %)'

In [45]:
loss = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text
loss

'961 (16.2 %)'

In [46]:
rating = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[1]/div/h6').text
rating

'2835'

In [47]:
driver.quit()

# Lo que hay que hacer (esquema)

In [48]:
# crea un codigo que haga lo que te pido a continuación:

# entrar en la url

url = 'https://www.yottachess.com/'

driver = webdriver.Chrome(PATH, options=opciones)

driver.get(url)

time.sleep(2)


#aceptar cookies

aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')

aceptar.click()

# introducir texto de busqueda, (habrá que hacer un loop para ir metiendo los nombres del top 100, los cuales estarán almacenados en una lista llamada 'top_100')

texto = driver.find_element(By.XPATH, '//*[@id="data"]')

texto.send_keys('Carlsen, Magnus')

# aprieta el boton enter del teclado

from selenium.webdriver.common.keys import Keys

texto.send_keys(Keys.ENTER)

#obtiene el porcentaje de victorias

win = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
win

#obtiene el porcentaje de empates

draw = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
draw

#obtiene el porcentaje de derrotas

losse = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text
losse

driver.quit()

# almacenar en un dataframe los datos obtenidos para cada jugador, es decir, tendrá que tener las siguentes columnas: nombre del jugador, wins, draws, losses (obtenidas en el paso anterior)

# realizar este proceso paralelizando con joblib.

# let's fucking go

In [59]:
# Función para obtener las estadísticas de un jugador
def obtener_estadisticas(jugador):
    driver = webdriver.Chrome(PATH, options=opciones)
    driver.get(url)
    time.sleep(2)

    # Aceptar cookies
    aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')
    aceptar.click()
    
    # Variables de asignación predeterminadas
    wins = 'N/A'
    draws = 'N/A'
    losses = 'N/A'
    
    try:
        # Introducir el nombre del jugador
        texto = driver.find_element(By.XPATH, '//*[@id="data"]')
        texto.send_keys(jugador)
        texto.send_keys(Keys.ENTER)
        time.sleep(2)

        # Obtener las estadísticas
        wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
        draws = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
        losses = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text

        driver.quit()  # Cerrar el navegador
        
    except:
        time.sleep(1)

    return {'Jugador': jugador, 'Wins': wins, 'Draws': draws, 'Losses': losses}

# URL y opciones del driver
url = 'https://www.yottachess.com/'
# PATH ya definido
opciones = webdriver.ChromeOptions()


# Lista de jugadores top 100
# top_100 ya creada

# Realizar el procesamiento en paralelo para obtener las estadísticas de los jugadores
paralelo = Parallel(n_jobs=3,  verbose=True)
resultados = paralelo(delayed(obtener_estadisticas)(jugador) for jugador in tqdm(top_100))


# Crear el DataFrame con los resultados
df = pd.DataFrame(resultados)

# Mostrar el DataFrame
df

  0%|          | 0/100 [00:00<?, ?it/s]

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 11.9min
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed: 29.0min finished


,Jugador,Wins,Draws,Losses
0,"Carlsen, Magnus",2879 (48.53 %),718 (35.27 %),961 (16.2 %)
1,"Caruana, Fabiano",2025 (42.75 %),547 (34.49 %),1078 (22.76 %)
2,"Nakamura, Hikaru",N/A,N/A,N/A
3,"Ding, Liren",760 (33.48 %),413 (47.4 %),434 (19.12 %)
4,"Firouzja, Alireza",1484 (52.68 %),223 (23.82 %),662 (23.5 %)
...,...,...,...,...
95,"Najer, Evgeniy",1181 (43.18 %),None (34.52 %),610 (22.3 %)
96,"Saric, Ivan",1153 (43.31 %),None (36.03 %),550 (20.66 %)
97,"Malakhov, Vladimir",N/A,N/A,N/A
98,"Liang, Awonder",N/A,N/A,N/A


In [70]:
df.columns = ['name', 'Wins', 'Draws', 'Losses']

In [71]:
print(df)

                          name            Wins           Draws          Losses
0              Carlsen, Magnus  2879 (48.53 %)   718 (35.27 %)    961 (16.2 %)
1             Caruana, Fabiano  2025 (42.75 %)   547 (34.49 %)  1078 (22.76 %)
2             Nakamura, Hikaru             N/A             N/A             N/A
3                  Ding, Liren   760 (33.48 %)    413 (47.4 %)   434 (19.12 %)
4            Firouzja, Alireza  1484 (52.68 %)   223 (23.82 %)    662 (23.5 %)
..                         ...             ...             ...             ...
95              Najer, Evgeniy  1181 (43.18 %)  None (34.52 %)    610 (22.3 %)
96                 Saric, Ivan  1153 (43.31 %)  None (36.03 %)   550 (20.66 %)
97          Malakhov, Vladimir             N/A             N/A             N/A
98              Liang, Awonder             N/A             N/A             N/A
99  Aravindh, Chithambaram VR.             N/A             N/A             N/A

[100 rows x 4 columns]


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Jugador  100 non-null    object
 1   Wins     100 non-null    object
 2   Draws    100 non-null    object
 3   Losses   100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


### guardado

In [76]:
df.to_csv(r'top_100_scraping.csv', index=False)

# Scrapeo de todos lo GM's

In [52]:
# Función para obtener las estadísticas de un jugador
def obtener_estadisticas(jugador):
    driver = webdriver.Chrome(PATH, options=opciones)
    driver.get(url)
    time.sleep(2)

    # Aceptar cookies
    aceptar = driver.find_element(By.XPATH, '//*[@id="ez-accept-necessary"]')
    aceptar.click()
    
    # Variables de asignación predeterminadas
    wins = 'N/A'
    draws = 'N/A'
    losses = 'N/A'
    
    try:
        # Introducir el nombre del jugador
        texto = driver.find_element(By.XPATH, '//*[@id="data"]')
        texto.send_keys(jugador)
        texto.send_keys(Keys.ENTER)
        time.sleep(2)

        # Obtener las estadísticas
        wins = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[9]/div/h6').text
        draws = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[10]/div/h6').text
        losses = driver.find_element(By.XPATH, '//*[@id="about"]/div[1]/div[11]/div/h6').text

        driver.quit()  # Cerrar el navegador
        
    except:
        time.sleep(1)

    return {'Jugador': jugador, 'Wins': wins, 'Draws': draws, 'Losses': losses}

# URL y opciones del driver
url = 'https://www.yottachess.com/'
# PATH ya definido
opciones = webdriver.ChromeOptions()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)

# Lista de jugadores top 100
# top_100 ya creada

# Realizar el procesamiento en paralelo para obtener las estadísticas de los jugadores
paralelo = Parallel(n_jobs=3,  verbose=True)
resultados = paralelo(delayed(obtener_estadisticas)(jugador) for jugador in tqdm(name))


# Crear el DataFrame con los resultados
df1 = pd.DataFrame(resultados)

# Mostrar el DataFrame
print(df1)

  0%|          | 0/1508 [00:00<?, ?it/s]

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


KeyboardInterrupt: 